In [1]:
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from datetime import datetime

## Configuration

In [2]:
VAR = 'discharge'
PATH = Path('../data/')
CLIMA_FILE = PATH / VAR / f'{VAR}_avg_1991-2020.nc'
YEAR_FILE = PATH / VAR / f'{VAR}_avg_2024.nc'
UPAREA_FILE = PATH / 'GloFAS_static_maps' / 'upArea.nc'
MIN_AREA = 5e8 # m2

In [3]:
map_variables = {
    'runoff': 'rowe',
    'discharge': 'dis24'
}

var_shortname = map_variables[VAR]

## Data

In [4]:
# GloFAS upstream area map
uparea = xr.open_dataset(UPAREA_FILE)['Band1']

In [5]:
# load climatology: average period 1991-2020
climatology = xr.open_dataset(CLIMA_FILE)[var_shortname]
climatology.close()

# load year of analysis: 2024
data = xr.open_dataset(YEAR_FILE)[var_shortname]
data.close()

## Anomaly

In [6]:
# compute annual anomaly
anomaly = (data - climatology) # m3/s

# correct name and coordinates
anomaly.name = VAR
anomaly = anomaly.drop_vars('surface', errors='ignore')

anomaly

<xarray.DataArray 'discharge' (lat: 3000, lon: 7200)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) float64 89.97 89.92 89.88 89.83 ... -59.87 -59.92 -59.98
  * lon      (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0

In [7]:
# remove pixels with small catchment area
uparea = uparea.sel(lat=slice(anomaly.lat.max(), anomaly.lat.min()))

In [8]:
mask_area = uparea >= MIN_AREA

mask_area.sum()

<xarray.DataArray 'Band1' ()>
array(1047616)

In [10]:
anomaly.where(mask_area)

<xarray.DataArray 'discharge' (lat: 3, lon: 3)>
array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) float64 89.97 89.92 89.88
  * lon      (lon) float64 -180.0 -179.9 180.0

In [ ]:
mask_area.sum()

In [ ]:
anomaly.isnull().all()

In [ ]:
# add attributes
anomaly.attrs['long_name'] = 'river discharge anomaly'
anomaly.attrs['units'] = 'm3/s'
anomaly.attrs['climatology'] = '1991-2020'
anomaly.attrs['source'] = 'GloFASv4'
anomaly.attrs['crs'] = 'epsg:4326'
anomaly.attrs['author'] = 'Jesús Casado Rodríguez <jesus.casado-rodriguez@ec.europa.eu>'
anomaly.attrs['institution'] = 'Joint Research Centre - European Commission'
anomaly.attrs['history'] = 'Created {0}'.format(datetime.now().strftime("%B %d %Y %H:%M:%S"))

### Plot

In [ ]:
# boundaries = [-1e3, -500, -250, -100, -50, -25, 25, 50, 100, 250, 500, 1e3]
# cmap = plt.cm.BrBG
# norm = mcolors.BoundaryNorm(boundaries, cmap.N)#, extend='both')

# # fig, ax = plt.subplots(figsize=(12, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))
# anomaly.plot(
#     cmap=cmap,
#     norm=norm,
#     cbar_kwargs={
#         'orientation': 'horizontal', 
#         'pad': 0.05, 
#         'label': 'Anomaly',
#         'ticks': [-500, -250, -100, -50, 25, 50, 100, 250, 500], 
#     }
# )
# ax.axis('off')

# plt.show()

In [ ]:
anomaly.isnull().all()

In [ ]:
anomaly.plot();

In [ ]:
# export
anomaly.to_netcdf(PATH / VAR / f'{VAR}_anomaly_mask.nc')